# Segmentez des clients d'un site e-commerce
## Notebook 2 : Analyse exploratoire et premier feature ingineering
OpenClassrooms - Parcours Data Scientist - Projet 05  

## Sommaire  
**Préparation de l'environnement**  
* Environnement virtuel
* Import des modules
* Fonction

**Analyse exploratoire**
* Schéma relationnel   
* Analyses univariées

**Premier feature engineering**
 


# 1 Préparation de l'environnement

## 1.1 Environnement virtuel

In [1]:
# Vérification environnement virtuel
envs = !conda env list
print(f"Environnement virtuel : {[e for e in envs if '*' in e][0].split('*')[1].strip()}")

Environnement virtuel : C:\Users\chrab\anaconda3\envs\opc5


## 1.2 Import des modules

In [2]:
# Installation des librairies
!pip install ipython-sql --quiet
!pip install pandas --quiet
!pip install numpy --quiet

In [3]:
# Import des modules
import pandas as pd
import sqlite3

## 1.3 Fonctions

In [4]:
# Renvoit le résultat d'une requête SQL
def query_result(query: str, all_rows: bool=True):
    conn = sqlite3.connect('olist.db')
    cursor = conn.cursor()
    cursor.execute(query)
    if all_rows:
        result = cursor.fetchall()
    else:
        result = cursor.fetchone()
        if len(result) == 1:
            result = result[0]
    return result

In [5]:
# Description des champs d'une table
def describe_table(table_name: str, fields=None):
    
    # Préparation des colonnes du dataframe de descriptions
    df_names, df_types, df_unique_values, df_duplicates, df_missing_values, df_percentage_missing = [], [], [], [], [], []
    
    # Récupération de la liste des champs de la table
    pragma = query_result(f"PRAGMA table_info({table_name});")
    table_fields = [row[1] for row in pragma]

    # Constitution de la liste des champs à décrire
    if fields is None:
        fields = [field for field in table_fields if field != 'index']
    elif isinstance(fields, str):
        fields = [fields]
        
    # Récupération/calculs des informations descriptives
    total_rows = query_result(f"SELECT COUNT(*) FROM {table_name};", False)
    for field_index, field_name in enumerate(table_fields):
        if field_name in fields:
            field_type = pragma[field_index][2]
            unique_values = query_result(f"SELECT COUNT(DISTINCT {field_name}) FROM {table_name};", False)
            non_null_values = query_result(f"SELECT COUNT({field_name}) FROM {table_name};", False)
            duplicates = non_null_values - unique_values
            missing_values = total_rows - non_null_values
            percentage_missing = (missing_values / total_rows) * 100 if total_rows > 0 else 0
            percentage_missing = str(round(percentage_missing, 2)) + ' %'

            df_names.append(field_name)
            df_types.append(field_type)
            df_unique_values.append(unique_values)
            df_duplicates.append(duplicates)
            df_missing_values.append(missing_values)
            df_percentage_missing.append(percentage_missing)

    # Céation du dataframe de description
    df_infos = pd.DataFrame({
        'Colonne': df_names,
        'Type': df_types,
        'Valeurs uniques': df_unique_values,
        'Doublons': df_duplicates,
        'Valeurs manquantes': df_missing_values,
        '% valeurs manquantes': df_percentage_missing
    }).reset_index(drop=True)
    
    return df_infos

# 2 Analyse exploratoire

## 2.1 Schéma relationnel

![shema relationnel](schema_relationnel_olist.png "Shéma relationnel BDD olist")

L'objectif est de réaliser une **segmentation des clients**.  
La première étape est de réaliser une RFM simple, je vais donc porter une attention particulière sur les champs me permettant de calculer les scores :  
* **Récence** : date de la dernière commande => `orders.order_approved_at` ?
* **Fréquence** : nombre total de commandes => `customers.customer_id`, `orders.order_id` ou `orders.customer_id` ?
* **Montant** : valeur totale des achats => `order_items.price` (+ `order_items.freight` ?), `order_pymts.payment_value` ?  

## 2.2 Analyses univariées

### 2.2.1 Table `orders`

In [6]:
display(describe_table('orders'))

,Colonne,Type,Valeurs uniques,Doublons,Valeurs manquantes,% valeurs manquantes
0,order_id,TEXT,99441,0,0,0.0 %
1,customer_id,TEXT,99441,0,0,0.0 %
2,order_status,TEXT,8,99433,0,0.0 %
3,order_purchase_timestamp,TEXT,98875,566,0,0.0 %
4,order_approved_at,TEXT,90733,8548,160,0.16 %
5,order_delivered_carrier_date,TEXT,81018,16640,1783,1.79 %
6,order_delivered_customer_date,TEXT,95664,812,2965,2.98 %
7,order_estimated_delivery_date,TEXT,459,98982,0,0.0 %
